In [1]:
from rx import Observable, Observer


In [2]:
from sqlalchemy import create_engine, text

In [3]:
from __future__ import print_function

In [14]:
import pprint
pp = pprint.PrettyPrinter(depth=6)

In [4]:
import requests


headers = {'Content-type': 'application/x-www-form-urlencoded'}

data="""    USE AstxDB;
select *  from TableBT v; 
"""

response = requests.post('http://45.79.91.219:19002/query/service', data=data, headers=headers)

info = {
    "url" : 'http://45.79.91.219:19002/query/service',
    "headers" : headers,
    "data" : data
}

x = [info]

source = Observable.from_(x)
source.subscribe(lambda x: print(x))

{'url': 'http://45.79.91.219:19002/query/service', 'headers': {'Content-type': 'application/x-www-form-urlencoded'}, 'data': '    USE AstxDB;\nselect *  from TableBT v; \n'}


In [6]:
response.text

u'{\n\t"requestID": "fd04aab7-b23b-452d-b5a3-2c6400e2012a",\n\t"signature": "*",\n\t"results": [ { "v": { "id": 1, "parent_id": 1, "childName": "Langdon Childress" } }\n, { "v": { "id": 2, "parent_id": 2, "childName": "George Cavanaugh" } }\n, { "v": { "id": 3, "parent_id": 2, "childName": "Susan Sarandon" } }\n, { "v": { "id": 4, "parent_id": 3, "childName": "Kris Sarah" } }\n ]\n\t,\n\t"status": "success",\n\t"metrics": {\n\t\t"elapsedTime": "447.646402ms",\n\t\t"executionTime": "437.852808ms",\n\t\t"resultCount": 4,\n\t\t"resultSize": 273\n\t}\n}\n'

In [7]:
import json
jsonobj = json.loads(response.text)

In [20]:
xml = '''
    <root>
        <PG processor="postgresql_exec" conf="./postgresql.conf">
            <SQLBook>
                <public>
                    <customers>
                        <COL NAME="customerid" TYPE="String" />
                        <COL NAME="firstname" TYPE="String" />
                    </customers>
                    <reviews>
                        <COL NAME="customerid" TYPE="String" />
                        <COL NAME="firstname" TYPE="String" />
                    </reviews>                    
                </public>
            </SQLBook>         
            <DB>
                <SCHEMA>
                    <VIEW1>
                        <COL NAME="Col1" TYPE="String" />
                        <COL NAME="Col2" TYPE="Integer" />
                    </VIEW1>
                    <VIEW2>
                        <COL NAME="Col1" TYPE="String" />
                        <COL NAME="Col2" TYPE="Integer" />
                    </VIEW2>                    
                </SCHEMA>
            </DB>
            <orm2>
                <public>
                    <AddressTable>
                        <COL NAME="Col1" TYPE="String" />
                        <COL NAME="Col2" TYPE="Integer" />
                    </AddressTable>
                </public>
            </orm2>
            <dbTest>
                <public>
                    <table1>
                        <COL NAME="id" TYPE="Integer" />
                        <COL NAME="name" TYPE="String" />
                    </table1>                  
                </public>
            </dbTest>             
        </PG>
        <AX processor="asterixdb_exec2" conf="./astrixdb.conf">
                <AstxDB>
                    <TableBT>
                        <COL NAME="id" TYPE="Integer" />
                        <COL NAME="childName" TYPE="String" />
                    </TableBT>                     
                </AstxDB>
            </AX>
        </root>
'''

In [21]:
pp.pprint(xml)

'\n    <root>\n        <PG processor="postgresql_exec" conf="./postgresql.conf">\n            <SQLBook>\n                <public>\n                    <customers>\n                        <COL NAME="customerid" TYPE="String" />\n                        <COL NAME="firstname" TYPE="String" />\n                    </customers>\n                    <reviews>\n                        <COL NAME="customerid" TYPE="String" />\n                        <COL NAME="firstname" TYPE="String" />\n                    </reviews>                    \n                </public>\n            </SQLBook>         \n            <DB>\n                <SCHEMA>\n                    <VIEW1>\n                        <COL NAME="Col1" TYPE="String" />\n                        <COL NAME="Col2" TYPE="Integer" />\n                    </VIEW1>\n                    <VIEW2>\n                        <COL NAME="Col1" TYPE="String" />\n                        <COL NAME="Col2" TYPE="Integer" />\n                    </VIEW2>   

In [22]:
from lxml import etree
root = etree.XML(xml)


#find_text = etree.XPath("//PG/DB/SCHEMA/View1/child::node()")
#print(root.find("PG/DB/SCHEMA")[0].tag)


In [315]:
import json
query = '''
{
    "from" : {
                "v1" : "PG/dbTest/public/table1",
                "v2" : "AX/AstxDB/TableBT"
            },
    "where" : "v1.id = v2.parent_id",
    "order" : "v1.name",
    "opts" : {
                "random" : {}
             },
    "return" : "v1.id, v2.parent_id, v1.name, v2.childName" 
}
'''
jl = json.loads(query)
jl

{u'from': {u'v1': u'PG/dbTest/public/table1', u'v2': u'AX/AstxDB/TableBT'},
 u'opts': {u'random': {}},
 u'order': u'v1.name',
 u'return': u'v1.id, v2.parent_id, v1.name, v2.childName',
 u'where': u'v1.id = v2.parent_id'}

In [316]:
viv = {x : { \
            "key" : x, \
            "db" : jl["from"][x], \
            "where" : jl["where"], \
            "return" : jl["return"] \
           } for x in jl["from"].keys()}

In [317]:
viv["v2"]

{'db': u'AX/AstxDB/TableBT',
 'key': u'v2',
 'return': u'v1.id, v2.parent_id, v1.name, v2.childName',
 'where': u'v1.id = v2.parent_id'}

In [318]:
import re

def getDomain(val):
    rec = re.compile(r'^([A-Z][A-Z])/')
    res = rec.findall(val)
    return ''.join(res)


getDomain(u'//AX/DB/SCHEMA/View1')

''

In [319]:
table = root.find('AX/AstxDB/TableBT').tag
table

'TableBT'

In [320]:
class SomeObserver(Observer):

    def on_next(self, value):

        processor = root.find(value["domain"]).attrib["processor"]
        
        if value["domain"] not in domain.keys():
            domain[value["domain"]] = {}
        domain[value["domain"]][value["key"]] = { 
            "processor" : processor,
            "db" : value["db"],
            "where" : value["where"],
            "return" : value["return"]
        }

    def on_completed(self):
        print("Done!")

    def on_error(self, error):
        print("Error Occurred: {0}".format(error))


In [321]:
domain={}
viv = {}
resultset = {}

Observable.from_(jl["from"].keys()) \
    .map(lambda x: {
                    "domain" : getDomain(jl["from"][x]),
                    "key" : x, "db" : jl["from"][x], \
                    "where" : jl["where"], "return" : jl["return"] \
                   }) \
    .subscribe(SomeObserver()) #lambda x: print(x))


Done!


In [322]:
import pprint
pp = pprint.PrettyPrinter(depth=6)
pp.pprint(domain)

{u'AX': {u'v2': {'db': u'AX/AstxDB/TableBT',
                 'processor': 'asterixdb_exec2',
                 'return': u'v1.id, v2.parent_id, v1.name, v2.childName',
                 'where': u'v1.id = v2.parent_id'}},
 u'PG': {u'v1': {'db': u'PG/dbTest/public/table1',
                 'processor': 'postgresql_exec',
                 'return': u'v1.id, v2.parent_id, v1.name, v2.childName',
                 'where': u'v1.id = v2.parent_id'}}}


In [323]:
import json
import requests

In [324]:
class SomeObserver3(Observer):
    
    def on_next(self, value):        
        targets = ([x[1] for x in [(value[2], x.strip()) for x in value[5].split(",")] if x[0] in x[1] ])
        
        argsdict = {
                    'alias' : value[2],
                    'where' : value[3],
                    'db' : value[4],
                    'ret' : targets
                   }

        globals()[value[0]](**argsdict)        

    def on_completed(self):
        pass

    def on_error(self, error):
        print("Error Occurred: {0}".format(error))     

class SomeObserver2(Observer):
    def on_next(self, value):
        Observable.from_(domain[value]) \
        .map(lambda x: (domain[value][x]['processor'],
                        value, 
                        x, 
                        domain[value][x]['where'], 
                        domain[value][x]['db'], 
                        domain[value][x]['return'])) \
        .subscribe(SomeObserver3())

    def on_completed(self):
        pass

    def on_error(self, error):
        print("Error Occurred: {0}".format(error))

In [325]:
def postgresql_exec(alias = None, where = None, db = None, ret = None):
    print("From PGsql")
    
    engine = create_engine('postgresql+psycopg2://postgres@45.79.91.219/dbTest')
    conn = engine.connect()   
    
    
    def runquery():
        realdb = (root.find(db + "/..").tag)
        table = root.find("{0}".format(db)).tag
        
        t = (','.join(ret))
        print(t)

        sql="""
        SELECT {0} FROM {1} {2}
        LIMIT 1000;
        """
        stmt = text(sql.format(t, table, alias))
                
        return Observable.from_(conn.execute(stmt))
    
    def myComplete():
        
        key1 = where.split('=')[0].strip()
        key2 = where.split('=')[1].strip()

        print (alias, key1)
        print(resultset)
        
        print(resultset[alias]['cols'].index(key1))

        colId1 = resultset[alias]['cols'].index(key1) 

        print (colId1)
        
        print ([x for x in resultset[alias]['rows']])

        keyids = set([x[colId1] for x in resultset[alias]['rows']])
        w=(' OR '.join(['{0} = {1}'.format(key2, x) for x in keyids]))
        asterixconfig['keys'] = w        
        
        
        print("finished")
        
    def myError(x):
        pass
    
    def save(x):
        if alias not in resultset.keys():
            resultset[alias] = {
                                    'cols' : ret,
                                    'rows' : []
                                   }
        resultset[alias]["rows"].append(x)


    runquery() \
        .subscribe(save, myError, myComplete)
        
    conn.close()
    
    # save the keys

In [326]:
def asterixdb_exec2(sql = None):
    headers = {'Content-type': 'application/x-www-form-urlencoded'}
    
#     response = requests.post('http://localhost:19002/query/service', data=sql, headers=headers) 
#     jsonobj = json.loads(response.text.replace("{ ,", "{ ")) 
    
#     def extractFields(x):
#         return tuple([x[field.replace(alias + ".","")] for field in ret])
    
#     def save(x):
#         if alias not in resultset.keys():
#             resultset[alias] = {
#                                     'cols' : ret,
#                                     'rows' : []
#                                }
#         resultset[alias]["rows"].append(x)        
    
#     Observable.from_(jsonobj["results"]) \
#     .map(lambda x: x[alias]) \
#     .map(extractFields) \
#     .subscribe(save)

In [327]:
resultset = {}

In [328]:
resultset

{}

In [329]:
w

'id = "1" OR id = "2" OR id = "3"'

In [330]:
asterixconfig={}

In [331]:
def asterixdb_exec2(alias = None, where = None, db = None, ret = None, keyids = None):
    headers = {'Content-type': 'application/x-www-form-urlencoded'}

    realdb = (root.find(db + "/..").tag)
    table = root.find("{0}".format(db)).tag
    
    t = (','.join(ret))
    
    print (db,t,alias)
    
    sqltemplate="""
    USE {0}
    select {2} from {1} {2}
    WHERE <0>;
    """
    asterixconfig['sql']=sqltemplate.format(realdb, table, alias)
    asterixconfig['ret']=ret
    asterixconfig['alias']=alias
    asterixconfig['db']=db
    asterixconfig['where']=where
    print (asterixconfig['sql'])
    

    

In [332]:
x={u'parent_id': 1, u'id': 1, u'childName': u'Langdon Childress'}
x

{u'childName': u'Langdon Childress', u'id': 1, u'parent_id': 1}

In [333]:
def asterixdb_exec3(alias = None, where = None, db = None, ret = None):
    sqlT = asterixconfig['sql']
    sqlT = sqlT.replace("<","{").replace(">","}")
    sql = sqlT.format(asterixconfig['keys'])
    print(sql)
    
    ret = asterixconfig['ret']
    alias = asterixconfig['alias']
    
    response = requests.post('http://45.79.91.219:19002/query/service', data=sql, headers=headers) 

    jsonobj = json.loads(response.text.replace("{ ,", "{ ")) 
    
    def extractFields(x):
#         print (x)
#         print tuple([x[field.replace(alias + ".","")] for field in ret])
#         return x
        return tuple([x[field.replace(alias + ".","")] for field in ret])
    
    def save(x):
        if alias not in resultset.keys():
            resultset[alias] = {
                                    'cols' : ret,
                                    'rows' : []
                               }
        resultset[alias]["rows"].append(x)        
    
    Observable.from_(jsonobj["results"]) \
    .map(lambda x: x[alias]) \
    .map(extractFields) \
    .subscribe(save)

In [334]:
resultset.keys()

[]

In [335]:
resultset={}
Observable.from_(domain) \
.subscribe(SomeObserver2())

AX/AstxDB/TableBT v2.parent_id,v2.childName v2

    USE AstxDB
    select v2 from TableBT v2
    WHERE <0>;
    
From PGsql
v1.id,v1.name
v1 v1.id
{u'v1': {'rows': [(1, u'Michael Jordan'), (2, u'Eric Gordon'), (3, u'Shangri La')], 'cols': [u'v1.id', u'v1.name']}}
0
0
[(1, u'Michael Jordan'), (2, u'Eric Gordon'), (3, u'Shangri La')]
finished


In [336]:
pp = pprint.PrettyPrinter(depth=6)
pp.pprint(resultset)

{u'v1': {'cols': [u'v1.id', u'v1.name'],
         'rows': [(1, u'Michael Jordan'),
                  (2, u'Eric Gordon'),
                  (3, u'Shangri La')]}}


In [337]:
asterixconfig

{'alias': u'v2',
 'db': u'AX/AstxDB/TableBT',
 'keys': 'v2.parent_id = 1 OR v2.parent_id = 2 OR v2.parent_id = 3',
 'ret': [u'v2.parent_id', u'v2.childName'],
 'sql': '\n    USE AstxDB\n    select v2 from TableBT v2\n    WHERE <0>;\n    ',
 'where': u'v1.id = v2.parent_id'}

In [338]:
asterixdb_exec3()


    USE AstxDB
    select v2 from TableBT v2
    WHERE v2.parent_id = 1 OR v2.parent_id = 2 OR v2.parent_id = 3;
    


In [339]:
resultset

{u'v1': {'cols': [u'v1.id', u'v1.name'],
  'rows': [(1, u'Michael Jordan'), (2, u'Eric Gordon'), (3, u'Shangri La')]},
 u'v2': {'cols': [u'v2.parent_id', u'v2.childName'],
  'rows': [(1, u'Langdon Childress'),
   (2, u'George Cavanaugh'),
   (2, u'Susan Sarandon'),
   (3, u'Kris Sarah')]}}

In [340]:
[x[field.replace(alias + ".","")] for field in ret]

[1, u'Langdon Childress']

In [341]:
asterixconfig

{'alias': u'v2',
 'db': u'AX/AstxDB/TableBT',
 'keys': 'v2.parent_id = 1 OR v2.parent_id = 2 OR v2.parent_id = 3',
 'ret': [u'v2.parent_id', u'v2.childName'],
 'sql': '\n    USE AstxDB\n    select v2 from TableBT v2\n    WHERE <0>;\n    ',
 'where': u'v1.id = v2.parent_id'}

In [342]:
key1 = asterixconfig['where'].split('=')[0].strip()
key2 = asterixconfig['where'].split('=')[1].strip()

In [343]:
aliases=resultset.keys()

In [344]:
resultset[aliases[0]]

{'cols': [u'v1.id', u'v1.name'],
 'rows': [(1, u'Michael Jordan'), (2, u'Eric Gordon'), (3, u'Shangri La')]}

In [345]:
resultset

{u'v1': {'cols': [u'v1.id', u'v1.name'],
  'rows': [(1, u'Michael Jordan'), (2, u'Eric Gordon'), (3, u'Shangri La')]},
 u'v2': {'cols': [u'v2.parent_id', u'v2.childName'],
  'rows': [(1, u'Langdon Childress'),
   (2, u'George Cavanaugh'),
   (2, u'Susan Sarandon'),
   (3, u'Kris Sarah')]}}

In [346]:
key2

u'v2.parent_id'

In [347]:
resultset

{u'v1': {'cols': [u'v1.id', u'v1.name'],
  'rows': [(1, u'Michael Jordan'), (2, u'Eric Gordon'), (3, u'Shangri La')]},
 u'v2': {'cols': [u'v2.parent_id', u'v2.childName'],
  'rows': [(1, u'Langdon Childress'),
   (2, u'George Cavanaugh'),
   (2, u'Susan Sarandon'),
   (3, u'Kris Sarah')]}}

In [348]:
colKeyIndex1 = resultset[aliases[0]]['cols'].index(key1)
colKeyIndex2 = resultset[aliases[1]]['cols'].index(key2)

for i, row1 in enumerate(resultset[aliases[0]]['rows']):
    row1Key = (row1[colId1])
    for j, row2 in enumerate(resultset[aliases[1]]['rows']):
        if row1[colKeyIndex1] == row2[colKeyIndex2]:
            print(row1, row2)

(1, u'Michael Jordan') (1, u'Langdon Childress')
(2, u'Eric Gordon') (2, u'George Cavanaugh')
(2, u'Eric Gordon') (2, u'Susan Sarandon')
(3, u'Shangri La') (3, u'Kris Sarah')
